In [1]:
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt

from deepface import DeepFace
import math

# Setting

In [2]:
metrics = ["cosine", "euclidean", "euclidean_l2"] # 유사도 설정 가능

workpath = "C:\\Users\\ksko\\Desktop\\work\\"
filepath = "C:\\Users\\ksko\\Desktop\\work\\w_deep\\"


src = "C:\\Users\\ksko\\Desktop\\work\\w_00010.jpg"

total_data = os.listdir(filepath)

filepath 에는 동영상이 들어있는 폴더 경로
workpath 에는 filepath의 전단계 경로

src에는 src로 비교할 이미지 한개 경로

최종적으로 total_data에 들어가는 경로가 동영상이 전부 들어있는 폴더입니다.

In [3]:
total_data

['num1_guide.mp4',
 'num1_reaction.mp4',
 'num1_task1_question.mp4',
 'num1_task1_reaction.mp4',
 'num1_task2_question.mp4',
 'num1_task2_reaction.mp4',
 'num1_task3_question.mp4',
 'num1_task3_reaction.mp4',
 'num1_task4_question.mp4',
 'num1_task4_reaction.mp4',
 'num1_task5_question.mp4',
 'num1_task5_reaction.mp4',
 'num1_title.mp4',
 'num2_description.mp4',
 'num2_guide.mp4',
 'num2_reaction.mp4',
 'num2_task1_question.mp4',
 'num2_task1_reaction.mp4',
 'num2_task2_question.mp4',
 'num2_task2_reaction.mp4',
 'num2_task3_question.mp4',
 'num2_task3_reaction.mp4',
 'num2_task4_question.mp4',
 'num2_task4_reaction.mp4',
 'todo_description.mp4',
 'todo_guide.mp4',
 'todo_reaction.mp4',
 'todo_task1_question.mp4',
 'todo_task1_reaction.mp4',
 'todo_task2_question.mp4',
 'todo_task2_reaction.mp4',
 'todo_task3_question.mp4',
 'todo_task3_reaction.mp4',
 '관계01_Mission_Complete_Reaction.mp4',
 '관계01_Mission_Start_Description_En.mp4',
 '관계01_Mission_Start_Guide_En.mp4',
 '관계01_Mission_Star

In [4]:
scenario = ['인사', '시험공부01','시험공부02','면접01','사랑01','전화01','비지니스01','대화와응답01',
            '의견파악01','관계01','todo','num1','num2']

# 숫자정보01 : num1
# 숫자정보02 : num2
# 할일,부탁할일01 : todo


#scenario = ['시험공부01','시험공부02','면접01','사랑01','전화01','비지니스01','대화와응답01',
#            '의견파악01','관계01','할일','숫자정보01','숫자정보02']

동영상 맨 앞 제목을 scenario list에 적어서 넣어주시면됩니다.

저의 경우 숫자정보01이 num1_~~~ 로 저장되어 있어 해당부분을 수정하여 넣어둔 상태입니다.

그러고 나서 아래 코드도 이름 맞춰 넣어주시면 됩니다.

In [11]:
# 영상 제목들을 넣을 리스트
greet = []
test01 = []
test02 = []
interview = []
love = []
call = []
business = []
conversation = []
opinion = []
relation = []
todo = []
num1 = []
num2 = []

# 영상 제목들을 추가
for i in range(len(total_data)):
    if total_data[i][:6] == '시험공부01':
        test01.append(total_data[i])
    elif total_data[i][:6] == '시험공부02':
        test02.append(total_data[i])
    elif total_data[i][:2] == '면접':
        interview.append(total_data[i])
    elif total_data[i][:2] == '사랑':
        love.append(total_data[i])
    elif total_data[i][:2] == '전화':
        call.append(total_data[i])
    elif total_data[i][:2] == '비지':
        business.append(total_data[i])
    elif total_data[i][:2] == '대화':
        conversation.append(total_data[i])
    elif total_data[i][:2] == '의견':
        opinion.append(total_data[i])
    elif total_data[i][:2] == '관계':
        relation.append(total_data[i])
    elif total_data[i][:4] == 'todo':
        todo.append(total_data[i])
    elif total_data[i][:4] == 'num1':
        num1.append(total_data[i])
    elif total_data[i][:4] == 'num2':
        num2.append(total_data[i])
    elif total_data[i][:2] == '인사': # 인사가 빠짐!!
        greet.append(total_data[i])


# 영상의 유사도를 저장하는 리스트
greet_result = []
test01_result = []
test02_result = []
interview_result = []
love_result = []
call_result = []
business_result = []
conversation_result = []
opinion_result = []
relation_result = []
todo_result = []
num1_result = []
num2_result = []


# 영상 제목을 넣는 리스트를 묶은 리스트 (이중리스트)
name_list = [greet, test01, test02, interview, love, 
            call, business, conversation, opinion, 
            relation, todo, num1, num2]

# 영상의 유사도를 저장하는 리스트를 묶은 리스트 (이중리스트)
result_list = [greet_result, test01_result, test02_result, interview_result, love_result,
              call_result, business_result, conversation_result, opinion_result,
              relation_result, todo_result, num1_result, num2_result]

In [12]:
# list(map(len, (test01, test02, interview, love, call, business, conversation, opinion, relation, todo, num1, num2)))
# 빈거 체크

# Code

In [13]:
# 시나리오별로 반복
#for i in range(len(scenario)):
for i in range(0, 1):
    print(f"--- 현재 진행중인 시나리오--- : {scenario[i]}")
    temp_name_list = name_list[i]
    temp_result_list = result_list[i]
    
    # 영상별로 반복
    for name in temp_name_list:
        path = filepath + name # 여자
        video = cv2.VideoCapture(path)
        if(video.isOpened()):
            print(f"현재 진행중인 영상 : {name}")
        length = video.get(cv2.CAP_PROP_FRAME_COUNT)
        cut = math.floor(length/10) # 구간을 10개로 나눈다
        if cut == 0: # 프레임이 1인경우 0으로 나누게 되어 오류가 남. 제외시키자
            continue
        count = 0
        
        print('영상의 프레임수', length)
        
        # 영상의 프레임별로 반복
        while(video.isOpened()):
            ret, img = video.read() # 다음 프레임 읽기
            
            if (int(video.get(1))%cut==0 & count <= 10): # 10개의 구간
                try:
                    result = DeepFace.verify(img1_path = img, # 유사도 측정
                              img2_path = src,
                              distance_metric = metrics[0],
                    )
                except: # 얼굴 인식이 안된경우 예외처리
                    count -= 1
                    pass
                count += 1                
                print(f'유사도 : {result["distance"]}\r', end='') # 유사도 출력
                # 11번째로 넘어가는 경우 추가하지 않는다.
                if count <= 10:
                    temp_result_list.append(result['distance'])
                else: count -= 1
                
            if video.get(1) >= length : 
                print("                              ", end='')
                print(f"\r나눠진 사진 수 : {count} \n")
                break
    # 완료된 시나리오에 대해 외부로 csv export            
    pd.Series(result_list[i]).to_csv(workpath + 'deep_w_' + scenario[i] + '.csv', index=False, header=False)

--- 현재 진행중인 시나리오--- : 인사
현재 진행중인 영상 : 인사_description.mp4
영상의 프레임수 172.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_guide.mp4
영상의 프레임수 104.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_reaction_8.mp4
영상의 프레임수 135.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_result.mp4
영상의 프레임수 107.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_task1_question.mp4
영상의 프레임수 103.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_task1_reaction.mp4
영상의 프레임수 99.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_task2_question.mp4
영상의 프레임수 72.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_task2_reaction.mp4
영상의 프레임수 78.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_task3_question.mp4
영상의 프레임수 75.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_task3_reaction.mp4
영상의 프레임수 65.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_task4_question.mp4
영상의 프레임수 102.0
나눠진 사진 수 : 10                 

현재 진행중인 영상 : 인사_task4_reaction.mp4
영상의 프레임수 61.0
나눠진 사진 수 : 10                 



In [14]:
for i in range(len(scenario)):
    print(f"{scenario[i]}의 유사도 평균은 {round(pd.Series(result_list[i]).mean(),2)}, 표준편차는 {round(pd.Series(result_list[i]).std(),2)} 입니다.")

인사의 유사도 평균은 0.26, 표준편차는 0.21 입니다.
시험공부01의 유사도 평균은 nan, 표준편차는 nan 입니다.
시험공부02의 유사도 평균은 nan, 표준편차는 nan 입니다.
면접01의 유사도 평균은 nan, 표준편차는 nan 입니다.
사랑01의 유사도 평균은 nan, 표준편차는 nan 입니다.
전화01의 유사도 평균은 nan, 표준편차는 nan 입니다.
비지니스01의 유사도 평균은 nan, 표준편차는 nan 입니다.
대화와응답01의 유사도 평균은 nan, 표준편차는 nan 입니다.
의견파악01의 유사도 평균은 nan, 표준편차는 nan 입니다.
관계01의 유사도 평균은 nan, 표준편차는 nan 입니다.
todo의 유사도 평균은 nan, 표준편차는 nan 입니다.
num1의 유사도 평균은 nan, 표준편차는 nan 입니다.
num2의 유사도 평균은 nan, 표준편차는 nan 입니다.
